In [28]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("song.txt")

paper = loader.load()

paper

[Document(metadata={'source': 'song.txt'}, page_content='â€¦ I had a dream\nI got everything I wanted\nNot what you\'d think\nAnd if I\'m being honest\nâ€¦ It might\'ve been a nightmare\nTo anyone who might care\nThought I could fly (fly)\nSo I stepped off the Golden, mm\nNobody cried (cried, cried, cried, cried)\nNobody even noticed\nI saw them standing right there\nKinda thought they might care (might care, might care)\nâ€¦ I had a dream\nI got everything I wanted\nBut when I wake up, I see\nYou with me\nâ€¦ And you say, "As long as I\'m here\nNo one can hurt you\nDon\'t wanna lie here\nBut you can learn to\nIf I could change\nThe way that you see yourself\nYou wouldn\'t wonder why you hear\nThey don\'t deserve you"\nâ€¦ I tried to scream\nBut my head was underwater\nThey called me weak\nLike I\'m not just somebody\'s daughter\nâ€¦ Coulda been a nightmare\nBut it felt like they were right there\nAnd it feels like yesterday was a year ago\nBut I don\'t wanna let anybody know\n\'Cause 

In [31]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
song_split = text_splitter.split_documents(paper)

song_split

[Document(metadata={'source': 'song.txt'}, page_content='â€¦ I had a dream\nI got everything I wanted\nNot what you\'d think\nAnd if I\'m being honest\nâ€¦ It might\'ve been a nightmare\nTo anyone who might care\nThought I could fly (fly)\nSo I stepped off the Golden, mm\nNobody cried (cried, cried, cried, cried)\nNobody even noticed\nI saw them standing right there\nKinda thought they might care (might care, might care)\nâ€¦ I had a dream\nI got everything I wanted\nBut when I wake up, I see\nYou with me\nâ€¦ And you say, "As long as I\'m here\nNo one can hurt you'),
 Document(metadata={'source': 'song.txt'}, page_content='No one can hurt you\nDon\'t wanna lie here\nBut you can learn to\nIf I could change\nThe way that you see yourself\nYou wouldn\'t wonder why you hear\nThey don\'t deserve you"\nâ€¦ I tried to scream\nBut my head was underwater\nThey called me weak\nLike I\'m not just somebody\'s daughter\nâ€¦ Coulda been a nightmare\nBut it felt like they were right there\nAnd it fe

In [32]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

paper_db = FAISS.from_documents(song_split, embedding=OllamaEmbeddings(model="llama3"))

In [33]:
query = "What dream did singer have?"
results = paper_db.similarity_search(query)
results[0].page_content

'What would they say instead?\nIf I knew it all then would I do it again?\nWould I do it again?\nIf they knew what they said would go straight to my head\nWhat would they say instead?'

In [34]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")

In [35]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the question based on the provided context.
    Think step by step before providing the answer.
    I will tip you $1000 if you answer correctly.
    <context>
    {context}
    </context>
    Question: {input}
    """
)

In [36]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [37]:
retriever=paper_db.as_retriever()
retriever  

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020BF825E630>, search_kwargs={})

In [38]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [39]:
answer = retrieval_chain.invoke({"input": "What dream did singer have?"})
answer['answer']

'What an intriguing question!\n\nAfter analyzing the context and lyrics, it seems that the singer had a dream of achieving everything they wanted. The line "I got everything I wanted" suggests that they reached their goals or desires.\n\nHowever, the song also hints at the darker side of this dream. The lines "It might\'ve been a nightmare / To anyone who might care" suggest that achieving one\'s dreams wasn\'t all it was cracked up to be. It\'s possible that the singer felt trapped or unhappy with their accomplishments.\n\nThe repeated references to having someone ("You") by their side and feeling protected ("As long as I\'m here, no one can hurt you") imply that the singer\'s dream may have been closely tied to personal relationships.\n\nSo, what is this dream? Based on the lyrics, it seems that the singer had a dream of achieving success and being loved or appreciated for who they are.'

In [40]:
answer   

{'input': 'What dream did singer have?',
 'context': [Document(id='02dcec98-b9a9-4349-96ba-d0fd8fb1efa0', metadata={'source': 'song.txt'}, page_content='What would they say instead?\nIf I knew it all then would I do it again?\nWould I do it again?\nIf they knew what they said would go straight to my head\nWhat would they say instead?'),
  Document(id='6d4eaaa0-0570-4d27-9e2b-dd0c81693602', metadata={'source': 'song.txt'}, page_content='And I don\'t wanna let \'em down\nâ€¦ I had a dream\nI got everything I wanted\nBut when I wake up, I see\nYou with me\nâ€¦ And you say, "As long as I\'m here\nNo one can hurt you\nDon\'t wanna lie here\nBut you can learn to\nIf I could change\nThe way that you see yourself\nYou wouldn\'t wonder why you hear\nThey don\'t deserve you"\nâ€¦ If I knew it all then would I do it again?\nWould I do it again?\nIf they knew what they said would go straight to my head\nWhat would they say instead?'),
  Document(id='273aa767-b93a-4323-824d-598a8898b3c8', metadata=